In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import axes
import pickle
from datetime import datetime
from scipy import log,exp,sqrt,stats
print('Packages')

Packages


In [2]:
# Importing pickle file
data1 = np.load('/Users/louisribieras/Desktop/Invictus/Data/latest_coin_data.pkl')
data2 = np.load('/Users/louisribieras/Desktop/Invictus/Data/additional_data.pkl')

# Removing July dates from data1
data1 = data1.drop(data1.columns[0:4], axis = 1)

# Combining the two dataframes
data = pd.concat([data2, data1], axis = 1, join = 'outer', sort = True)

# Filling Nan values with 0
data = data.dropna()

# Saving original date index
date_index = data.columns

print('Loading of Data')
print('shape of data: ', data.shape)

Loading of Data
shape of data:  (127, 947)


In [3]:
print('Getting Market Caps')
# Initialize a close_price list
mc_list = list()
# Making a list of all close_prices
for date in data.index:
    for column in data.columns:
        mc_list.append(data[column][date]['market_cap'])

Getting Market Caps


In [4]:
print('Making Market Cap dataframe')
# Splitting list into 133 lists (one per coin)
mc_list = [mc_list[i:i+947] for i  in range(0, len(mc_list), 947)]   
# Making dataframe
mc_data = pd.DataFrame(data = mc_list, index = data.index, columns = data.columns)

Making Market Cap dataframe


In [5]:
print('Getting Close Prices')
# Initialize a close_price list
cp_list = list()
# Making a list of all close_prices
for date in data.index:
    for column in data.columns:
        cp_list.append(data[column][date]['close_price'])

Getting Close Prices


In [6]:
print('Making Close Price dataframe')
# Splitting list into 133 lists (one per coin)
cp_list = [cp_list[i:i+947] for i  in range(0, len(cp_list), 947)]   
# Making dataframe
close_data = pd.DataFrame(data = cp_list, index = data.index, columns = data.columns)

Making Close Price dataframe


In [7]:
print('flipping the dates in order')
# Reording the date index
close_data = close_data[close_data.columns[::-1]]
mc_data = mc_data[mc_data.columns[::-1]]
data = data[data.columns[::-1]]

flipping the dates in order


In [8]:
print('Share cap functions')

def share_cap_recurse(market_cap_list, num_coins, cap_ceiling_amount, final_cap_list):

    not_realloc_val = 0

    realloc_val = 0

    num_final = 0

    new_market_cap_list = []

    

    for i in range(0, len(market_cap_list)):

        if(market_cap_list[i] > cap_ceiling_amount):

            final_cap_list.append(cap_ceiling_amount)

            realloc_val += market_cap_list[i] - cap_ceiling_amount

            num_final += 1

        else:

            not_realloc_val += market_cap_list[i]

     

    if(num_final!=0):

        new_market_cap_list = [x * (1+(realloc_val/not_realloc_val)) for x in market_cap_list[num_final:]]

    else:

        new_market_cap_list = deepcopy(market_cap_list)

    

    if len(final_cap_list) == num_coins:

        return

    elif (new_market_cap_list[0] <= cap_ceiling_amount): #assumes ordered desc in market cap

        for cap in new_market_cap_list:

            final_cap_list.append(cap)

        return       

    else:

        share_cap_recurse(new_market_cap_list, num_coins, cap_ceiling_amount, final_cap_list)

        

def share_cap(market_cap_list, cap_ceiling_percent):  #CHANGE CAP CEILING HERE

    total_cap = sum(market_cap_list)

    cap_ceiling_amount = total_cap * cap_ceiling_percent

    final_cap_list = []

    

    share_cap_recurse(market_cap_list, len(market_cap_list), cap_ceiling_amount, final_cap_list)

    

    return (final_cap_list)

Share cap functions


In [9]:
# Selecting dates and median market caps
selectedmarket = mc_data.copy()
selectedmarket = selectedmarket.loc[:, 'Jul 01, 2017':'Jul 01, 2018':7]
# selectedmarket = selectedmarket.resample('W',axis=1).median()

# Selecting dates and median close price
selectedclose = close_data.copy()
selectedclose = selectedclose.loc[:, 'Jul 01, 2017':'Jul 01, 2018':7]
# selectedclose = selectedclose.resample('W',axis=1).median()

# Daily close prices
dailyclose = close_data.loc[:,'Jan 01, 2017':'Jan 01, 2018']
dailyclose1 = dailyclose.copy().T
dailychanget = dailyclose1.pct_change(1).T
dailychange = dailyclose1.pct_change(1).T

# Calculating percent change for returns
selectedclose1 = selectedclose.copy().T
selectedreturns = selectedclose1.pct_change(1).T
selectedreturns.head()
print('selected market, selectedclose and selectedreturns')

selected market, selectedclose and selectedreturns


In [10]:
print('Total market cap movement for tethershare')
tmc_list = list()

for column in selectedmarket.columns:
    tmc_list.append(selectedmarket[column].sum())
    
tmc_data = pd.DataFrame(data = tmc_list, columns = ['total_mc'])
tmc_data.index = selectedmarket.columns
tmc_pct = tmc_data.pct_change(1)

# Resampling the tmc data to get weekly
tmc_pct = tmc_pct.loc['Jul 01, 2017':'Jun 30, 2018',:]
# tmc_pct = tmc_pct.resample('W').median()
tmc_pct_index = tmc_pct.index

tmc_pct.head(5)

Total market cap movement for tethershare


,total_mc
"Jul 01, 2017",NaN
"Jul 08, 2017",-0.056833
"Jul 15, 2017",-0.152925
"Jul 22, 2017",0.140825
"Jul 29, 2017",-0.010164


In [11]:
num_index = np.arange(53)
tmc_pct.index = num_index

usdt_share = {}
usdt_share[0] = 0.10      #USDT starts off with ___% for the date


for i in tmc_pct.index[1:]:
    if tmc_pct['total_mc'][i] < -0.01:
        usdt_share[i] = usdt_share[i -1] - tmc_pct['total_mc'][i]
        if usdt_share[i] >= 0.5:
            usdt_share[i] = 0.5
    elif tmc_pct['total_mc'][i] > 0.01: 
        usdt_share[i] = usdt_share[i -1] - tmc_pct['total_mc'][i]
        if usdt_share[i] <= 0:
            usdt_share[i] = 0
    else:
        usdt_share[i] = usdt_share[i -1]

In [12]:
ilist = range(0, len(tmc_pct))

tethershare = {}
         
for (i, column) in zip(ilist, selectedmarket):
    tethershare[column] = usdt_share[i]
         
print('setting tethershare labels to match dates')

setting tethershare labels to match dates


In [13]:
# Calculating the returns on the tether held
usdt_close = pd.DataFrame(data = dailyclose.loc['tether'])
usdt_returns = usdt_close.pct_change(1)

In [14]:
print('loop for C10 with tether market cap function')
#FOR LOOP FOR C10 with tether
topmc10t = {}
sc10t = {}
tc10t = {}
ps10t = {}
invest10t = {}
returns10t = {}
lg_returns10t = {}
selectedmc10t = {}
topcoins10t = {}
c10sharet = {}
tetherreturns = {}

daily_changet = {}
weighted_daily_changet = {}
daily_returnst = {}
final_daily_returnst = list()


#for every date
for column in selectedmarket:
    #sort in descending order
    selectedmc10t[column] = selectedmarket[column].sort_values(ascending=False)
    
    #take the top coins
    topmc10t[column] = selectedmc10t[column].head(10)
    #take the index of the top coins
    topcoins10t[column] = topmc10t[column].index
    
    #get the share cap of the top 10
    sc10t[column] = share_cap(topmc10t[column], 0.15)      #CHOICE OF CAP CEILING
    
    #total cap of top 10
    tc10t[column] = sum(sc10t[column])
    
    #percentage of total fund allocated to tether/C10- NEW CODE!!
    tetherfund = 0                    #INSERT CODE TO DETERMINE % OF FUNDS ALLOCATED TO TETHERFUND
    c10fund = 1 - tetherfund
    
    #percentage share to be allocated for top 10 - NEW CODE!!!
    ps10t[column] = [x * c10fund / tc10t[column] for x in sc10t[column] ]
    
    #building dataframe of top20 coins by mc and %share to be invested
    invest10t[column] = pd.DataFrame()
    invest10t[column] = pd.DataFrame({'coin': topcoins10t[column], 'pct_share': ps10t[column]})
    invest10t[column] = invest10t[column].set_index('coin')
    
jlist = list(range(0,365,7))
    
for (j, column) in zip(jlist, selectedmarket):
    daily_changet[column] = pd.DataFrame()
    daily_changet[column] = pd.DataFrame(data = dailychanget.iloc[:,j:j+7])
    daily_changet[column] = daily_changet[column].loc[topcoins10t[column]]
    weighted_daily_changet[column] = daily_changet[column].mul(ps10t[column], axis=0)
    
    daily_returnst[column] = weighted_daily_changet[column].sum()
    daily_returnst[column].columns = ['daily_returns']
        
    final_daily_returnst.extend(daily_returnst[column])

loop for C10 with tether market cap function


In [18]:
C10t_daily_returns = pd.DataFrame(data = final_daily_returnst)
C10t_daily_returns.columns = ['daily_returns']
C10t_daily_returns['date'] = dailychanget.T.index
C10t_daily_returns = C10t_daily_returns.set_index('date')
C10t_daily_returns.index = pd.to_datetime(C10t_daily_returns.index)

In [19]:
print('loop for C10 daily without tether')
#FOR LOOP FOR C10 without tether
topmc10 = {}
sc10 = {}
tc10 = {}
ps10 = {}
invest10 = {}
returns10 = {}
lg_returns10 = {}
selectedmc10 = {}
topcoins10 = {}
c10share = {}
tetherreturns = {}

daily_change = {}
weighted_daily_change = {}
daily_returns = {}
final_daily_returns = list()


#for every date
for column in selectedmarket:
    #sort in descending order
    selectedmc10[column] = selectedmarket[column].sort_values(ascending=False)
    
    #take the top coins
    topmc10[column] = selectedmc10[column].head(10)
    #take the index of the top coins
    topcoins10[column] = topmc10[column].index
    
    #get the share cap of the top 10
    sc10[column] = share_cap(topmc10[column], 0.15)      #CHOICE OF CAP CEILING
    
    #total cap of top 10
    tc10[column] = sum(sc10[column])
    
    #percentage share to be allocated for top 20 - NEW CODE!!!
    ps10[column] = [x  / tc10[column] for x in sc10[column] ]
    
    #building dataframe of top20 coins by mc and %share to be invested
    invest10[column] = pd.DataFrame()
    invest10[column] = pd.DataFrame({'coin': topcoins10[column], 'pct_share': ps10[column]})
    invest10[column] = invest10[column].set_index('coin')
    
jlist = list(range(0,365,7))
    
for (j, column) in zip(jlist, selectedmarket):
    daily_change[column] = pd.DataFrame()
    daily_change[column] = pd.DataFrame(data = dailychange.iloc[:,j:j+7])
    daily_change[column] = daily_change[column].loc[topcoins10[column]]
    weighted_daily_change[column] = daily_change[column].mul(ps10[column], axis=0)
    
    daily_returns[column] = weighted_daily_change[column].sum()
    daily_returns[column].columns = ['daily_returns']
        
    final_daily_returns.extend(daily_returns[column])

loop for C10 daily without tether


In [20]:
C10_daily_returns = pd.DataFrame(data = final_daily_returnst)
C10_daily_returns.columns = ['daily_returns']
C10_daily_returns['date'] = dailychanget.T.index
C10_daily_returns = C10_daily_returns.set_index('date')
C10_daily_returns.index = pd.to_datetime(C10_daily_returns.index)

In [22]:
C10t_daily_returns

,daily_returns
date,
2017-01-01,0.000000
2017-01-02,0.022455
2017-01-03,0.047690
2017-01-04,0.106072
2017-01-05,-0.089737
2017-01-06,-0.073374
2017-01-07,0.006785
2017-01-08,0.004827
2017-01-09,0.014050
